In [39]:
# Loading data from csv file
import pandas as pd
import numpy as np


In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [41]:
movies=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Movie/tmdb_5000_movies.csv")
movies=movies[['title','genres','keywords','overview']]
movies.rename(columns={'title':"name"},inplace=True)
movies.head(2)

,name,genres,keywords,overview
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."


In [44]:
movies.columns=['zdff','hbioo','nipo','njo']

In [48]:
movies.columns.values

array(['zdff', 'hbioo', 'nipo', 'njo'], dtype=object)

In [ ]:
credit=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Movie/tmdb_5000_credits.csv")
credit.rename(columns={'title':'name','movie_id':'id'},inplace=True)
credit.head(2)

,id,name,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
print(movies.shape)
print(credit.shape)

(4803, 4)
(4803, 4)


#Data Cleaning:

In [ ]:
# Merging on "name" columns
movies = movies.merge(credit,on='name')

In [ ]:
movies.shape

(4809, 7)

In [ ]:
#checking duplicate rows
movies.duplicated().sum()

0

In [ ]:
# checking null values in each column 
movies.isnull().sum()

name        0
genres      0
keywords    0
overview    3
id          0
cast        0
crew        0
dtype: int64

In [ ]:
# dropping rows with NA values
movies.dropna(inplace=True)
movies.isnull().sum()

name        0
genres      0
keywords    0
overview    0
id          0
cast        0
crew        0
dtype: int64

In [ ]:
movies.sample(3)

,name,genres,keywords,overview,id,cast,crew
3150,Amour,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 428, ""name"": ""nurse""}, {""id"": 2176, ""n...",Georges and Anne are in their eighties. They a...,86837,"[{""cast_id"": 6, ""character"": ""Anne"", ""credit_i...","[{""credit_id"": ""580f95e9c3a3683a46000c19"", ""de..."
3579,Oculus,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 3298, ""name"": ""hallucination""}, {""id"":...",A woman tries to exonerate her brother's murde...,157547,"[{""cast_id"": 3, ""character"": ""Kaylie Russell"",...","[{""credit_id"": ""53c10889c3a3684cdb006b6c"", ""de..."
1304,Sgt. Bilko,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 2952, ""name"": ""gulf war""}, {""id"": 4595...",The army is known for churning out lean mean f...,9099,"[{""cast_id"": 6, ""character"": ""Master Sergeant ...","[{""credit_id"": ""52fe44dbc3a36847f80ae3d7"", ""de..."


In [ ]:
import re

In [ ]:
#gen_0=movies.genres[0]
#gen_0='_ '+gen_0
#print(gen_0)

In [ ]:
#print(re.sub(r"\W+|_", " ", gen_0))

# Extracting generes keywords cast and crew

In [ ]:
import ast

In [ ]:
def _help(lst,val):# val=genres,keyword,actor,director
  genres=[]
  keywords=[]
  director=[]
  actor=[]
  #-------------------------------------------------------
  if val=="actor":
    for dic in lst[0:3]:
      actor.append(dic["name"].lower().replace(" ",""))
    return actor
  else:
    for dic in lst:
      #--------------------genres-------------------------
      if val=="genres":
        genres.append(dic["name"].lower().replace(" ",""))
      #------------------Keywords---------------------------
      elif val=="keyword":
        keywords.append(dic["name"].lower().replace(" ",""))
      #-----------------crew----------------------------------
      elif val=="director":
        if dic["job"]=="Director":
          director.append(dic["name"].lower().replace(" ",""))
  #-----------------------------------------------------------
  if val=="genres":
    return genres
  elif val=="keyword":
    return keywords
  elif val=="director":
    return director

In [ ]:
def tag(genres,keyword,cast,crew,overview):
  rows=[]
  # which runs on genres,keywords,cast and crew
  for i in range(len(genres)):
    temp=overview[i].lower()
    #-----------------genres-------------
    gen=ast.literal_eval(genres[i])
    A=_help(gen,"genres")
    temp=temp+" ".join(A)+" "
    #---------------Keyword---------------
    key=ast.literal_eval(keyword[i])
    B=_help(key,"keyword")
    temp=temp+" ".join(B)+" "
    #--------------cast---------------
    cst=ast.literal_eval(cast[i])
    C=_help(cst,"actor")
    temp=temp+" ".join(C)+" "
    #----------------Crew----------------
    cre=ast.literal_eval(crew[i])
    D=_help(cre,"director")
    temp=temp+" ".join(D)+""
    #---------------------------------------
    rows.append(re.sub(r"\W+|_", " ", temp))
    del temp,A,B,C,D,gen,key,cst,cre
  return np.array(rows)

In [ ]:
#calling Tag() function that returns the numpy array that contains "numpy_strings"
X=tag(movies.genres.values,movies.keywords.values,movies.cast.values,movies.crew.values,movies.overview.values)

# Adding "X" as column named "Tag", to movies DataFrame
movies["Tags"]=X

# Dropping columns [genres', 'keywords', 'overview','cast','crew']
movies.drop(columns=['genres', 'keywords', 'overview','cast','crew'],inplace=True)
movies.head(2)

,name,id,Tags
0,Avatar,19995,in the 22nd century a paraplegic marine is dis...
1,Pirates of the Caribbean: At World's End,285,captain barbossa long believed to be dead has ...


# converting "Tags",to Vecotors, using TFIDF


**Performing Stemming, before convering into vectors**

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def stemmer(tags):
  PS=PorterStemmer()
  rows=[]
  for sen in tags:
    #---------------------
    lst=word_tokenize(sen)
    for i in range(len(lst)):
      lst[i]=PS.stem(lst[i])
    rows.append((" ").join(lst))
    del lst
  return np.array(rows)  

In [ ]:
# now, X is stemmed
X=stemmer(movies.Tags)

**TF IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_vec=TfidfVectorizer(max_features=5000,stop_words='english',ngram_range=(1,1),min_df=10)
data=X_vec.fit_transform(X)

In [ ]:
# Feature Names
#X_vec.get_feature_names_out()

In [ ]:
X_vec.idf_

array([5.7642564 , 5.81426682, 6.14562396, ..., 6.99292182, 7.0799332 ,
       6.83877114])

In [ ]:
len(X_vec.get_feature_names_out())

3292

**Calculating Cosine Similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
CS=cosine_similarity(data)

In [ ]:
CS.shape

(4806, 4806)

In [ ]:
CS[0]

array([1.        , 0.02588031, 0.03502353, ..., 0.05077461, 0.00629113,
       0.        ])

In [ ]:
len(CS[0])

4806

**Movie Recommender**

In [ ]:
import random

In [ ]:
def recommend_movie(movie_name):
  idx=movies[movies.name==movie_name].index[0]
  temp=np.argsort(CS[idx])[::-1][1:6]
  #print(CS[0][temp])
  #print("-"*40)
  print("#. "+movie_name)
  print("5 recommended movies below:")
  for i in temp:
    print(movies["name"].iloc[i])
  print("-"*40)

In [ ]:
recommend_movie("Harry Potter and the Philosopher's Stone")

#. Harry Potter and the Philosopher's Stone
5 recommended movies below:
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Half-Blood Prince
Harry Potter and the Order of the Phoenix
Harry Potter and the Prisoner of Azkaban
----------------------------------------


In [ ]:
r=random.sample(range(0,len(list(movies.name))),10)
for idx in r:
  recommend_movie(list(movies.name)[idx])

#. The East
5 recommended movies below:
Boat Trip
The Broken Hearts Club: A Romantic Comedy
The Bubble
Never Again
Milk
----------------------------------------
#. Babylon A.D.
5 recommended movies below:
Sheena
The Book of Eli
Men of War
The Collection
Mad Max
----------------------------------------
#. Step Up Revolution
5 recommended movies below:
Case 39
Flashdance
Take the Lead
Hide and Seek
Desert Dancer
----------------------------------------
#. Hocus Pocus
5 recommended movies below:
The Lords of Salem
The Last Witch Hunter
The Sisterhood of Night
Fun Size
The Craft
----------------------------------------
#. Jefferson in Paris
5 recommended movies below:
Like Crazy
Déjà Vu
Guiana 1838
The Tourist
Real Women Have Curves
----------------------------------------
#. Bleeding Hearts
5 recommended movies below:
Four Brothers
Stranded
A Streetcar Named Desire
The Last House on the Left
10th & Wolf
----------------------------------------
#. Keeping Up with the Steins
5 recommended m

#-----------------------------------------------

In [ ]:

#temp=re.compile(r'\d')# match only digits
#temp=re.compile(r'\D')# match everything except digits
#temp=re.compile(r'\s')# match only white space
#temp=re.compile(r'\S')# match everything except white space
#temp=re.compile(r'\w')# match only digits and alphabets and "underscore _"
#temp=re.compile(r'\W')# match only all special characters
#temp=temp.finditer(gen_0)
#for i in temp:
  #print(i)

In [147]:
df=pd.DataFrame({'col':['123.33','233.23','333.122',"?0.2_"]})
print(df)
print()
#-----------custom dunction-------------------------------
import re
def remove(string):
  temp=[]
  for S in string.split("."):
    v=re.sub(r"\W|_","",S)
    temp.append(v)
  return float(".".join(temp))
#--------------------------------------------------------
df['col'].apply(remove)


       col
0   123.33
1   233.23
2  333.122
3    ?0.2_



0    123.330
1    233.230
2    333.122
3      0.200
Name: col, dtype: float64

In [112]:

#rint("I have {} coins, each of 5₹: ".format(five_coins))
#print("I have {} coins, each of 1₹: ".format(one_coins))
amount_to_be_paid=int(input("Enter Amount to be paid: "))
five_coins=int(input("Enter no of 5₹ coins you have: "))
one_coins=int(input("Enter no of 1₹ coins you have: "))
print()
#-------------------------------------------------
total_amount=five_coins*5+one_coins*1
if total_amount<amount_to_be_paid:
  print(-1)

elif amount_to_be_paid%5==0:
  #4print(amount_to_be_paid%5)
  no_of_5rs_coins_needed=amount_to_be_paid/5
  no_of_1_rs_coins_needed=0
  print("No of 1₹ coins needed to make payment: "+str(int(no_of_1_rs_coins_needed)))
  print("No of 5₹ coins needed to make payment: "+str(int(no_of_5rs_coins_needed)))

elif amount_to_be_paid<five_coins*5:
  mod=amount_to_be_paid%5
  no_of_5_rs_coins_needed=int((amount_to_be_paid-mod)/5)
  no_of_1_rs_coins_needed=mod
  print("No of 1₹ coins needed to make payment: "+str(int(no_of_1_rs_coins_needed)))
  print("No of 5₹ coins needed to make payment: "+str(int(no_of_5_rs_coins_needed)))
else:
  no_of_1_rs_coins_needed=amount_to_be_paid-(five_coins*5)
  no_of_5_rs_coins_needed=five_coins
  print("No of 1₹ coins needed to make payment: "+str(int(no_of_1_rs_coins_needed)))
  print("No of 5₹ coins needed to make payment: "+str(int(no_of_5_rs_coins_needed)))


Enter Amount to be paid: 15
Enter no of 5₹ coins you have: 4
Enter no of 1₹ coins you have: 4

No of 1₹ coins needed to make payment: 0
No of 5₹ coins needed to make payment: 3
